In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
#from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [89]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [90]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [91]:
def weighted_random_recommendation(items_weights, n=5):
    """
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """    
    items = np.array(items_weights['item_id'])
    weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=weights)     
    return recs.tolist()

In [92]:
df = data[['item_id', 'quantity']].sort_values(by=['quantity'])
df = df.groupby(by=["item_id"]).sum().reset_index()
df.rename(columns={'quantity': 'weight'}, inplace=True)
m = df['weight'].sum()
df.loc[df['weight'] > 0, 'weight'] = df['weight'] / m

In [93]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [94]:
%%time

result['random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(df, n=5))
result.head(2)
# your_code

Wall time: 6.94 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6534178, 6534166, 6533889, 5747420, 989075]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534178, 6533889, 6534166, 13777060, 6544236]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [95]:
row = pd.read_csv('predictions_basic.csv')
row.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [96]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [106]:
row['actual'][0].replace("[", "").replace("]", "").replace(",", "").replace("  ", " ").replace("  ", " ").split(" ")

['',
 '821867',
 '834484',
 '856942',
 '865456',
 '889248',
 '907957',
 '914190',
 '943316\n',
 '951954',
 '954486',
 '958046',
 '962568',
 '969231',
 '971585',
 '979707',
 '986947\n',
 '990656',
 '995242',
 '1004906',
 '1005186',
 '1042083',
 '1050310',
 '1060819',
 '1062002\n',
 '1064441',
 '1069733',
 '1074612',
 '1082185',
 '1131115',
 '1132771',
 '6534544',
 '13876341\n',
 '15971874',
 '17178953',
 '883616',
 '917704',
 '931860',
 '961554',
 '1002032',
 '1031190\n',
 '8090541',
 '8293439',
 '9297615',
 '9527329',
 '15926712',
 '1049998',
 '861272',
 '869465\n',
 '877373',
 '908213',
 '933913',
 '940947',
 '945809',
 '959316',
 '978974',
 '1031697\n',
 '1041796',
 '1048918',
 '1081189',
 '1101422',
 '1115576',
 '1122428',
 '1132231',
 '1132814\n',
 '5577022',
 '8091601',
 '9296986',
 '9677939',
 '10356149',
 '13417048',
 '15741823',
 '15830875']

In [102]:
precision_at_k(row['random_recommendation'][0].replace("[", "").replace("]", "").replace(" ", "").split(","), row['random_recommendation'][0].replace("[", "").replace("]", "").replace(" ", "").split(","))

1.0

In [85]:
result.apply(lambda row: precision_at_k(np.array(row['random_recommendation']), np.array(row['actual']), 5), axis=1).mean()

AttributeError: 'numpy.ndarray' object has no attribute 'tocsr'

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code